In [3]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and

In [5]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c5d20ecb242f0e0ae51b951191627bd3e4ba71389eef398b59822ec56dbf3e12
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [6]:
import torch
import numpy as np
import evaluate
from sklearn.metrics.pairwise import cosine_similarity

# Hugging Face metrics
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
exact = evaluate.load("exact_match")

def evaluate_custom_encoder_decoder(
    df,
    encoder,            # your encoder model: text → embedding
    decoder,            # your decoder model: embedding → text
    llama_vectorizer,   # precomputed LLaMA emoji vectors (for ground truth)
    decoder_labels,     # ground truth decoded texts (e.g., emoji captions)
    use_tokenizer=False,
    tokenizer=None
):
    predictions = []
    references = []
    cosine_scores = []

    for idx, row in df.iterrows():
        input_text = row["text"]
        target_vector = llama_vectorizer[idx]           # from LLaMA
        target_text = decoder_labels[idx]               # demojized text version

        # Encode: text → embedding
        with torch.no_grad():
            embedding = encoder(input_text)
            if isinstance(embedding, tuple):  # if encoder returns (hidden, pooled)
                embedding = embedding[0]

        # Decode: embedding → text
        if use_tokenizer and tokenizer:
            # decoder is a LM like T5
            input_tokens = tokenizer(input_text, return_tensors="pt").to(embedding.device)
            decoded_ids = decoder.generate(**input_tokens)
            decoded_text = tokenizer.decode(decoded_ids[0], skip_special_tokens=True)
        else:
            decoded_text = decoder(embedding)

        predictions.append(decoded_text)
        references.append(target_text)

        # Cosine similarity between encoder output and LLaMA vector
        sim = cosine_similarity(
            embedding.detach().cpu().numpy().reshape(1, -1),
            target_vector.reshape(1, -1)
        )[0][0]
        cosine_scores.append(sim)

    # HF Metrics
    results = {
        "BLEU": bleu.compute(predictions=predictions, references=[[r] for r in references])["bleu"],
        "METEOR": meteor.compute(predictions=predictions, references=references)["meteor"],
        "ROUGE-L": rouge.compute(predictions=predictions, references=references)["rougeL"],
        "Exact Match": exact.compute(predictions=predictions, references=references)["exact_match"],
        "Cosine Similarity (Encoder)": float(np.mean(cosine_scores))
    }

    print("\n🧪 Evaluation Results (Custom Pipeline):")
    for k, v in results.items():
        print(f"{k}: {v:.4f}")

    return results


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
results = evaluate_custom_encoder_decoder(
    df=my_data_df,                            # must contain "text" column
    encoder=my_encoder,                       # your encoder wrapper
    decoder=my_decoder,                       # your decoder wrapper
    llama_vectorizer=llama_vectors,           # (N, D) embeddings from LLaMA
    decoder_labels=emoji_text_descriptions,   # ground truth captions
    use_tokenizer=False,                      # decoder is not HF model
    tokenizer=None
)
